In [1]:
import pandas as pd
from datasets import load_dataset
from datasets import Dataset
import json

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("/home/sagemaker-user/qwen3-function-calling-ft-conversational", use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    "/home/sagemaker-user/qwen3-function-calling-ft-conversational",
    trust_remote_code=True,
)

OSError: Error no file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory /home/sagemaker-user/qwen3-function-calling-ft-conversational.

In [3]:
import os
print(os.path.abspath("qwen3-function-calling-ft-conversational"))


/home/sagemaker-user/qwen3-function-calling-ft-conversational


In [4]:
model

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2560)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): lora.Linear8bitLt(
            (base_layer): Linear8bitLt(in_features=2560, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Identity()
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2560, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear8bitLt(
            (base_layer): Linear8bitLt(in_features=2560, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
              (defau

In [5]:
import json
import re

def extract_tool_calls(text):
    # Extract tool calls using regex
    tool_call_pattern = r"<tool_call>(.*?)</tool_call>"
    matches = re.findall(tool_call_pattern, text, re.DOTALL)

    # Parse each match as JSON
    extracted_calls = []
    for match in matches:
        try:
            extracted_calls.append(json.loads(match))
        except json.JSONDecodeError:
            print(f"Failed to decode JSON: {match}")
            return False

    return extracted_calls

def compare_tool_calls(predicted, actual):
    actual_tool_calls = json.loads(actual)
    return actual_tool_calls == predicted

In [6]:
test_df = pd.read_csv("./data/test_dataset.csv")

In [7]:
test_df = Dataset.from_pandas(test_df)

In [8]:
SYSTEM_PROMPT = "You are a helpful assistant that uses the provided tools to answer."

In [ ]:
# Initialize a list to collect results
results_list = []

# Iterate through the test_df
for index, row in enumerate(test_df):
    query = row['query']
    tools = json.loads(row['tools'])
    expected_calls = row['answers'])

        # Serialize each tool call's arguments to a JSON string
    # for call in expected_calls:
    #     if isinstance(call.get("arguments"), dict):
    #         call["arguments"] = json.dumps(call["arguments"])

    messages = [
        {"role" : "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": query}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tools=tools,
        tokenize=False,
        add_generation_prompt=True,
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # Generate model output
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=16384, 
        temperature=0.0,
        do_sample=False
    )
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
    predicted_text = tokenizer.decode(output_ids, skip_special_tokens=True)
    print("predicted_text", predicted_text)

    # Extract tool calls from predicted text
    predicted_calls = extract_tool_calls(predicted_text)
    print("extracted_calls", predicted_calls)

    # Compare extracted calls with expected calls
    match = compare_tool_calls(predicted_calls, expected_calls)

    # Append results to the list
    results_list.append({
        "query": query,
        "predicted_text": predicted_text,
        "extracted_calls": predicted_calls,
        "expected_calls": expected_calls,
        "match": match
    })

# Convert results list to a DataFrame
results_df = pd.DataFrame(results_list)

# Display the resulting DataFrame
results_df.head()

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


predicted_text </tool_call>
{"name": "circle_area", "arguments": {"radius": 6}}
</tool_call>
<tool_call>
{"name": "circle_area", "arguments": {"radius": 10}}
</tool_call>
extracted_calls [{'name': 'circle_area', 'arguments': {'radius': 10}}]
predicted_text </tool_call>
{"name": "calculate_investment_return", "arguments": {"initial_amount": 25000, "interest_rate": 0.0375, "num_years": 9}}
</tool_call>
extracted_calls []
